In [ ]:
'''
Here we are directly using VectorStoreIndex with faiss vector search but we can also use this with pinecone database go to the docs for studying.
'''

In [7]:
import faiss

# dimensions of llama3.1 with ollama
d = 4096
nlist = 10 # how many cells or how many voronoi cells
quantizer = faiss.IndexFlatL2(d)
faiss_index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [2]:

import os
from llama_index.core import (
    SimpleDirectoryReader,
    Settings,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.llms.groq import Groq
from IPython.display import Markdown, display
from dotenv import load_dotenv

load_dotenv()

e:\Codes\Data Sciene\AI\FAISS_Vector_Seach\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# load documents
documents = SimpleDirectoryReader("./extracted_output").load_data()

In [4]:
documents

[Document(id_='767316d7-b74a-40e0-8df0-784b74ce9064', embedding=None, metadata={'file_path': 'e:\\Codes\\Data Sciene\\AI\\FAISS_Vector_Seach\\extracted_output\\Vedant Rajpurohit Resume new.txt', 'file_name': 'Vedant Rajpurohit Resume new.txt', 'file_type': 'text/plain', 'file_size': 6306, 'creation_date': '2024-09-19', 'last_modified_date': '2024-09-19'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='AWS Cloud\r\nAn individual who possesses a Bachelor of Technology degree in Computer Science Specialized with Machine learning and AI. Completed a hand-full of internships and worked as an teaching profession in IT field. Seeking a career in data science and Generative AI in a challenging environment where I can utilize my technical skills towards the deve

In [5]:
Settings.embed_model = ollama_embedding = OllamaEmbedding(
    model_name="llama3.1",
    base_url="http://localhost:11434"
)
Settings.llm = Groq(model="llama-3.1-70b-versatile", api_key=os.environ.get("GROQ_API_KEY"))

In [9]:
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

RuntimeError: Error in virtual void __cdecl faiss::IndexIVFFlat::add_core(idx_t, const float *, const idx_t *, const idx_t *, void *) at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\IndexIVFFlat.cpp:52: Error: 'is_trained' failed

In [30]:
# save index to disk
index.storage_context.persist()

In [10]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
index = load_index_from_storage(storage_context=storage_context)

In [15]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("What research paper he has published")

In [16]:
display(Markdown(f"{response}"))

The research paper published is on "Human Computer interaction with detection of hand gesture to improve artistic creativity".